In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wbdata
import plotly.graph_objects as go
import random

# Define datasets to retrieve from the World Bank API
indicators = {'AG.LND.FRST.ZS':'Natural Resources', 'NY.GDP.MKTP.CD':'GDP','SP.POP.TOTL':'Population'}

# Build Country List
countries = wbdata.get_country(display=False)  

country_list = []
country_names = []
color_list = []
i = 0

for country in countries:
    if country['region']['value'] != 'Aggregates':
        country_list.append(country['id'])
        country_names.append(country['name'])
        color_list.append(random.randint(1,255))
        
country_df = pd.DataFrame(country_list)

# Retrieve World Bank data
data = wbdata.get_dataframe(indicators,country=country_list,data_date=False,convert_date=True,keep_levels=True)

# Aggregate the data by Country
last_df = pd.DataFrame(data.groupby('country').agg(
    {
        'Natural Resources':'max',
        'GDP':'max',
        'Population':'max'  
    })).reset_index()


# Filter out bad data (report later)
last_df = last_df.fillna(0)
last_df = last_df[(last_df['Natural Resources'] > 0)]

# Build the Bubble Chart
area = 500
x = last_df['GDP']
y = last_df['Natural Resources']
z = last_df['Natural Resources']

sizeref = 2.*max(z)/(100**2)

# Create figure
fig = go.Figure(data=[go.Scatter(
    x=x, y=y,
    mode='markers',
    #text=hover_text,
    marker=dict(
        color=color_list,
        size=z,
        sizemode='area',
        sizeref=sizeref,
        sizemin=4)
)])

# Tune marker appearance and layout
fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                              sizeref=sizeref, line_width=2))

fig.update_layout(
    title='Natural Resources',
    showlegend=True,
    xaxis=dict(
        title='GDP',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Forests',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)
fig.show()


# Save Figure
#fig.savefig("CO2 Chart.png")